In [791]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")

In [792]:
elo_data = pd.read_sql_query(
    "Select DISTINCT * From Elo_AllMatches",
    con=devengine,)


In [793]:
elo_data['Elo_Diff']=elo_data['Elo_Fav_Elo']-elo_data['Elo_Dog_Elo']
elo_data=elo_data[((elo_data['Elo_Diff']).lt(100))!=True]
elo_data['Winner']=elo_data.apply(lambda x: 'Elo_Fav' if x['Elo_Fav']==x['Winner'] else 'Elo_Dog',axis=1)
#elo_data['Winner_Odds']=elo_data.apply(lambda x: x['Elo_Fav_Odds'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Odds'],axis=1)
#elo_data['Winner_Est_Odds']=elo_data.apply(lambda x: x['Elo_Fav_Est_Odds'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Est_Odds'],axis=1)
#elo_data['Winner_Elo']=elo_data.apply(lambda x: x['Elo_Fav_Elo'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Elo'],axis=1)
elo_data['Fav_Odds']=elo_data[['Elo_Fav_Odds','Elo_Dog_Odds']].min(axis=1)
#elo_data=elo_data[elo_data['Elo_Fav_Odds']<1.8]
train_data=elo_data[elo_data['Date']<'2022-09-01']
test_data=elo_data[elo_data['Date']>='2022-09-01']
test_data_resulted=test_data.reset_index()
test_data=test_data.drop(columns=['Surface','Date','Sex','Elo_Fav','Elo_Dog','Resulted','Time'])
train_data=train_data.drop(columns=['Surface','Date','Sex','Elo_Fav','Elo_Dog','Resulted','Time'])


In [794]:
print(len(test_data))
print(len(test_data_resulted))

846
846


In [795]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [796]:
X = train_data.drop(["Winner"], axis=1)
y = train_data["Winner"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
model2 = LogisticRegression(max_iter=20000000)
#model2 = SVC()
model2.fit(X_train, y_train)
train_score2 = model2.score(X_train, y_train)
test_score2 = model2.score(X_test, y_test)
print(train_score2,test_score2)

0.7075228497773611 0.7040913415794482


In [797]:
model2.intercept_

array([-0.00144301])

In [798]:
pred2=model2.predict(test_data.drop(columns='Winner',axis=1))

In [799]:
len(test_data_resulted)

846

In [800]:
cols = [
    "Date",
    "Prediction",
    "Winner",
    "Elo_Fav",
    "Elo_Fav_Odds",
    "Elo_Dog",
    "Elo_Dog_Odds","Time","Resulted"
    # "Training_Accuracy",
    # "Testing_Accuracy",
]
df = pd.DataFrame(columns=cols)
List = []

In [801]:
for index, tuples in test_data_resulted.iterrows():
        #if index <> len(test_data_resulted):

            values = [
                test_data_resulted["Date"][index],
                pred2[index],
                test_data_resulted["Winner"][index],
                test_data_resulted["Elo_Fav"][index],
                test_data_resulted["Elo_Fav_Odds"][index],
                test_data_resulted["Elo_Dog"][index],
                test_data_resulted["Elo_Dog_Odds"][index],
                test_data_resulted["Time"][index],
                test_data_resulted["Resulted"][index],
                #  "{:.0%}".format(train_score2),
                #  "{:.0%}".format(test_score2),
            ]
            zipped = zip(cols, values)
            a_dictionary = dict(zipped)
            List.append(a_dictionary)

In [802]:
temp = pd.DataFrame(List)
df = pd.concat([df, temp])

In [803]:
df
df[df['Winner']==df['Prediction']]

,Date,Prediction,Winner,Elo_Fav,Elo_Fav_Odds,Elo_Dog,Elo_Dog_Odds,Time,Resulted
3,2022-09-01,Elo_Fav,Elo_Fav,Ludmilla Samsonova,1.55,Leylah Fernandez,2.49,None,None
6,2022-09-01,Elo_Fav,Elo_Fav,Daniil Medvedev,1.02,Arthur Rinderknech,16.20,None,None
7,2022-09-01,Elo_Fav,Elo_Fav,Karen Khachanov,1.23,Thiago Monteiro,4.25,None,None
8,2022-09-01,Elo_Fav,Elo_Fav,Yibing Wu,1.30,Nuno Borges,3.64,None,None
9,2022-09-01,Elo_Fav,Elo_Fav,Nick Kyrgios,1.12,Benjamin Bonzi,6.48,None,None
...,...,...,...,...,...,...,...,...,...
841,2022-10-16,Elo_Dog,Elo_Dog,Jurij Rodionov,2.06,Mattia Bellucci,1.72,00:00,True
842,2022-10-16,Elo_Fav,Elo_Fav,Yibing Wu,1.21,Li Tu,4.07,16:00,False
843,2022-10-16,Elo_Fav,Elo_Fav,Felix Auger Aliassime,1.46,Lorenzo Musetti,2.71,05:00,True
844,2022-10-16,Elo_Fav,Elo_Fav,Andrey Rublev,1.48,Sebastian Korda,2.69,21:30,False


In [804]:
exclude_today=df[df['Date']!='2022-10-14'].copy()

In [825]:
exclude_today['Predicton_Odds']=exclude_today.apply(lambda x: x['Elo_Fav_Odds'] if 'Elo_Fav' == x['Prediction'] else x['Elo_Dog_Odds'],axis=1)
#exclude_today['Winner_Odds']=exclude_today['Winner_Odds'].astype(float)
exclude_today_odds=exclude_today[((exclude_today['Predicton_Odds']).ge(1.5))&((exclude_today['Predicton_Odds']).le(1.7))].copy()
#exclude_today

In [826]:
len(exclude_today_odds)

186

In [827]:
len(exclude_today_odds[exclude_today_odds['Winner']==exclude_today_odds['Prediction']])/len(exclude_today_odds)

0.521505376344086

In [807]:
today=df[df['Date']>'2022-10-15'].copy()
today['Predicton_Odds']=today.apply(lambda x: x['Elo_Fav_Odds'] if 'Elo_Fav' == x['Prediction'] else x['Elo_Dog_Odds'],axis=1)
#today=today[((today['Predicton_Odds']).le(1.8))&((today['Predicton_Odds']).ge(1))]

In [808]:
exclude_today_odds

,Date,Prediction,Winner,Elo_Fav,Elo_Fav_Odds,Elo_Dog,Elo_Dog_Odds,Time,Resulted,Predicton_Odds
122,2022-09-07,Elo_Fav,Elo_Fav,Stuart Parker,1.96,Bu Yunchaokete,1.79,None,None,1.96
188,2022-09-13,Elo_Fav,Elo_Fav,Terence Atmane,1.91,Ben Patael,1.83,None,None,1.91
372,2022-09-25,Elo_Fav,Elo_Fav,Terence Atmane,1.98,Bu Yunchaokete,1.74,None,None,1.98
419,2022-09-27,Elo_Dog,Elo_Dog,Mitchell Krueger,1.78,Paul Jubb,1.96,None,None,1.96
655,2022-10-08,Elo_Fav,Elo_Dog,Fabian Marozsan,1.93,Lukas Klein,1.81,None,None,1.93
698,2022-10-11,Elo_Dog,Elo_Fav,Federico Gaio,1.72,Kyrian Jacquet,2.00,None,None,2.00
709,2022-10-11,Elo_Dog,Elo_Dog,Tung lin Wu,1.73,Kimmer Coppejans,2.02,None,None,2.02


In [809]:
len(today[today['Winner']==today['Prediction']])/len(today)

0.7073170731707317

In [817]:
today[today['Resulted']!='True'].sort_values(by='Time',ascending=True)

,Date,Prediction,Winner,Elo_Fav,Elo_Fav_Odds,Elo_Dog,Elo_Dog_Odds,Time,Resulted,Predicton_Odds
824,2022-10-16,Elo_Fav,Elo_Fav,Iga Swiatek,1.34,Jessica Pegula,3.30,07:10,False,1.34
816,2022-10-16,Elo_Fav,Elo_Dog,Danielle Rose Collins,1.60,Donna Vekic,2.36,09:55,False,1.60
835,2022-10-16,Elo_Fav,Elo_Fav,Max Purcell,1.05,Seita Watanabe,7.83,11:30,False,1.05
806,2022-10-16,Elo_Fav,Elo_Dog,Makoto Ochi,1.38,Yusuke Takahashi,2.80,11:30,False,1.38
834,2022-10-16,Elo_Fav,Elo_Fav,Maximilian Neuchrist,1.05,Toshihide Matsui,7.88,13:00,False,1.05
833,2022-10-16,Elo_Fav,Elo_Dog,Yuta Shimizu,1.35,Moon Lee Jea,2.96,14:30,False,1.35
832,2022-10-16,Elo_Fav,Elo_Fav,Keegan Smith,1.41,Cheong Eui Kim,2.70,16:00,False,1.41
842,2022-10-16,Elo_Fav,Elo_Fav,Yibing Wu,1.21,Li Tu,4.07,16:00,False,1.21
829,2022-10-16,Elo_Fav,Elo_Fav,Francesco Maestrelli,1.39,Filip Misolic,2.87,18:00,False,1.39
830,2022-10-16,Elo_Fav,Elo_Fav,Francesco Passaro,1.24,Lorenzo Giustino,3.85,18:00,False,1.24


In [811]:
today['Predicton_Odds']=today.apply(lambda x: x['Elo_Fav_Odds'] if 'Elo_Fav' == x['Prediction'] else x['Elo_Dog_Odds'],axis=1)
